In [35]:
import os
import pandas as pd
import requests
import json
import requests
import multiprocessing as mp
from random import randint
from time import sleep

import sys
sys.path.insert(0,'/global/project/projectdirs/metatlas/anaconda/lib/python2.7/site-packages' )
from rdkit import Chem
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem import rdMolDescriptors
from metatlas import metatlas_objects as metob

In [ ]:
metatlas_df = pd.read_pickle('/project/projectdirs/openmsi/projects/ben_run_pactolus/unique_compounds.pkl')
# df = pd.read_csv('/project/projectdirs/openmsi/projects/compound_data/jgi_molecules/new_jgi_compounds.csv')
# df.rename(columns = {'monoisotopoic_mw':'monoisotopic_mw'},inplace=True)

In [ ]:
msmls_df = pd.read_csv('ID_MetID_Jamboree.csv')
pubchem_ids = msmls_df.PC_CID.tolist()

In [ ]:
# df = df.convert_objects(convert_numeric=True)
# df.keys()

In [127]:
def write_pubchem_info_to_file(out_dir,inchi_key):
# https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/%d/property/InChI/json
    suffix = '.json'
    fname = os.path.join(out_dir, inchi_key + suffix)
    if not os.path.isfile(fname):
        url = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/inchikey/%s/synonyms/json'%inchi_key
        response = requests.get(url)
        try: 
            with open(fname, 'w') as fid:
                json.dump(response.json(), fid)
        except:
            print "could not query", inchi_key

def get_inchi_from_pubchem_cid(out_dir,cid):
# 
    suffix = '.json'
    fname = os.path.join(out_dir, str(cid) + suffix)
    if not os.path.isfile(fname):
        url = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/%d/property/InChI/json'%cid
        response = requests.get(url)
        try: 
            with open(fname, 'w') as fid:
                json.dump(response.json(), fid)
        except:
            print "could not query", inchi_key
            
def get_pubchem_url_from_inchikey(inchi_key):    
    url = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/inchikey/%s/cids/txt'%inchi_key
    response = requests.get(url)
    return '; '.join(['https://pubchem.ncbi.nlm.nih.gov/compound/%s'%s for s in response.text.split()])


In [120]:
get_pubchem_url_from_inchikey('GLDQAMYCGOIJDV-UHFFFAOYSA-N')


u'https://pubchem.ncbi.nlm.nih.gov/compound/19'

In [111]:
r.text

u'19\n'

In [95]:
%system rm /global/homes/b/bpb/inchi_cid_msmls/*.json

[]

In [ ]:
out_dir = '/global/homes/b/bpb/inchi_cid_msmls/'
for cid in pubchem_ids:
    print cid
    get_inchi_from_pubchem_cid(out_dir,cid)

In [98]:
df = pd.DataFrame({'cid':pubchem_ids[:-4]})

In [99]:
for i,row in df.iterrows():
    suffix = '.json'
    fname = os.path.join(out_dir, str(row.cid) + suffix)
    with open(fname, 'r') as fid:
        d = json.load(fid)
        df.loc[i,'inchi'] = d['PropertyTable']['Properties'][0]['InChI']


In [100]:
df.head()

,cid,inchi
0,19.0,InChI=1S/C7H6O4/c8-5-3-1-2-4(6(5)9)7(10)11/h1-...
1,47.0,"InChI=1S/C6H10O3/c1-3-4(2)5(7)6(8)9/h4H,3H2,1-..."
2,51.0,"InChI=1S/C5H6O5/c6-3(5(9)10)1-2-4(7)8/h1-2H2,(..."
3,58.0,"InChI=1S/C4H6O3/c1-2-3(5)4(6)7/h2H2,1H3,(H,6,7)"
4,70.0,"InChI=1S/C6H10O3/c1-4(2)3-5(7)6(8)9/h4H,3H2,1-..."


In [101]:
import_compounds = reload(import_compounds)

In [102]:
import structure_cleaning as import_compounds
def make_mol_from_smiles_and_inchi(row):
    mol= []
    if row['inchi']:
        mol = Chem.MolFromInchi(str(row['inchi']))#.encode('utf-8'))
    if mol:
        return mol

df.ROMol = df.apply(make_mol_from_smiles_and_inchi, axis=1)

def desalt_compounds_in_dataframe(x):
    if x:
        if x.GetNumAtoms()>1:
            c = import_compounds.desalt(x)
            if c[1]:
                return c[0]
            else:
                return x
    
df.ROMol = df.ROMol.apply(desalt_compounds_in_dataframe)


import_compounds = reload(import_compounds)
def neutralize_compounds_in_dataframe(x):
    if x:
        if x.GetNumAtoms()> 0:
            neutral_mol = []
            try:
                c = import_compounds.NeutraliseCharges(x)
                neutral_mol = c[0]
            except:
                pass
            if neutral_mol:
                return neutral_mol
            
df.ROMol = df.ROMol.apply(neutralize_compounds_in_dataframe)

def calculate_num_radicals_in_dataframe(x):
    num_radicals = 0.0
    if x:
        num_radicals = Descriptors.NumRadicalElectrons(x)        
    return num_radicals

def calculate_formula_in_dataframe(x):
    formula = ''
    if x:
        formula = rdMolDescriptors.CalcMolFormula(x)
    return formula

def calculate_monoisotopic_mw_in_dataframe(x):
    mw = 0.0
    if x:
        mw = Descriptors.ExactMolWt(x)
    return mw

def calculate_inchi_in_dataframe(x):
    inchi = ''
    if x:
        try:
            inchi = Chem.MolToInchi(x)
        except:
            pass#This fails when can't kekulize mol
    return inchi

def calculate_flattened_inchi_in_dataframe(x):
    flattened_inchi = ''
    if x:
        sm = Chem.MolToSmiles(x).replace('@','')
        flattened_rdkit_mol = Chem.MolFromSmiles(sm)
        try:
            flattened_inchi = Chem.MolToInchi(flattened_rdkit_mol)
        except:
            pass#This fails when can't kekulize mol
    return flattened_inchi

def calculate_inchikey_in_dataframe(x):
    ik = ''
    if x:
        try:
            ik = Chem.InchiToInchiKey(x)
        except:
            pass#This fails when can't kekulize mol.  Carbo-cations are the culprit usually. 
    return ik

def calculate_charge_in_dataframe(x):
    if x:
        my_charge = Chem.GetFormalCharge(x)
        return my_charge
            
df['charge'] = df.ROMol.apply(calculate_charge_in_dataframe)
df['formula'] = df.ROMol.apply(calculate_formula_in_dataframe)
df['monoisotopic_mw'] = df.ROMol.apply(calculate_monoisotopic_mw_in_dataframe)
df['M+proton'] = df['monoisotopic_mw'] + 1.007276
df['M+ammonium'] = df['monoisotopic_mw'] + 18.033823
df['M+sodium'] = df['monoisotopic_mw'] + 22.989218
df['M-proton'] = df['monoisotopic_mw'] - 1.007276
df['M+acetate'] = df['monoisotopic_mw'] + 59.013851
df['M+chloride'] = df['monoisotopic_mw'] + 34.969402

df['num_radicals'] = df.ROMol.apply(calculate_num_radicals_in_dataframe)

df['metatlas_inchi'] = df.ROMol.apply(calculate_inchi_in_dataframe)
df['metatlas_inchi_key'] = df.metatlas_inchi.apply(calculate_inchikey_in_dataframe)

df['flat_inchi'] = df.ROMol.apply(calculate_flattened_inchi_in_dataframe)
df['flat_inchikey'] = df.flat_inchi.apply(calculate_inchikey_in_dataframe)


In [103]:
df['pubchem_url_as_given'] = df.cid.apply(lambda x: 'https://pubchem.ncbi.nlm.nih.gov/compound/%d'%x)
df.head()

,cid,inchi,charge,formula,monoisotopic_mw,M+proton,M+ammonium,M+sodium,M-proton,M+acetate,M+chloride,num_radicals,metatlas_inchi,metatlas_inchi_key,flat_inchi,flat_inchikey,pubchem_url_as_given
0,19.0,InChI=1S/C7H6O4/c8-5-3-1-2-4(6(5)9)7(10)11/h1-...,0,C7H6O4,154.026609,155.033885,172.060432,177.015827,153.019333,213.040460,188.996011,0,InChI=1S/C7H6O4/c8-5-3-1-2-4(6(5)9)7(10)11/h1-...,GLDQAMYCGOIJDV-UHFFFAOYSA-N,InChI=1S/C7H6O4/c8-5-3-1-2-4(6(5)9)7(10)11/h1-...,GLDQAMYCGOIJDV-UHFFFAOYSA-N,https://pubchem.ncbi.nlm.nih.gov/compound/19
1,47.0,"InChI=1S/C6H10O3/c1-3-4(2)5(7)6(8)9/h4H,3H2,1-...",0,C6H10O3,130.062994,131.070270,148.096817,153.052212,129.055718,189.076845,165.032396,0,"InChI=1S/C6H10O3/c1-3-4(2)5(7)6(8)9/h4H,3H2,1-...",JVQYSWDUAOAHFM-UHFFFAOYSA-N,"InChI=1S/C6H10O3/c1-3-4(2)5(7)6(8)9/h4H,3H2,1-...",JVQYSWDUAOAHFM-UHFFFAOYSA-N,https://pubchem.ncbi.nlm.nih.gov/compound/47
2,51.0,"InChI=1S/C5H6O5/c6-3(5(9)10)1-2-4(7)8/h1-2H2,(...",0,C5H6O5,146.021523,147.028799,164.055346,169.010741,145.014247,205.035374,180.990925,0,"InChI=1S/C5H6O5/c6-3(5(9)10)1-2-4(7)8/h1-2H2,(...",KPGXRSRHYNQIFN-UHFFFAOYSA-N,"InChI=1S/C5H6O5/c6-3(5(9)10)1-2-4(7)8/h1-2H2,(...",KPGXRSRHYNQIFN-UHFFFAOYSA-N,https://pubchem.ncbi.nlm.nih.gov/compound/51
3,58.0,"InChI=1S/C4H6O3/c1-2-3(5)4(6)7/h2H2,1H3,(H,6,7)",0,C4H6O3,102.031694,103.038970,120.065517,125.020912,101.024418,161.045545,137.001096,0,"InChI=1S/C4H6O3/c1-2-3(5)4(6)7/h2H2,1H3,(H,6,7)",TYEYBOSBBBHJIV-UHFFFAOYSA-N,"InChI=1S/C4H6O3/c1-2-3(5)4(6)7/h2H2,1H3,(H,6,7)",TYEYBOSBBBHJIV-UHFFFAOYSA-N,https://pubchem.ncbi.nlm.nih.gov/compound/58
4,70.0,"InChI=1S/C6H10O3/c1-4(2)3-5(7)6(8)9/h4H,3H2,1-...",0,C6H10O3,130.062994,131.070270,148.096817,153.052212,129.055718,189.076845,165.032396,0,"InChI=1S/C6H10O3/c1-4(2)3-5(7)6(8)9/h4H,3H2,1-...",BKAJNAXTPSGJCU-UHFFFAOYSA-N,"InChI=1S/C6H10O3/c1-4(2)3-5(7)6(8)9/h4H,3H2,1-...",BKAJNAXTPSGJCU-UHFFFAOYSA-N,https://pubchem.ncbi.nlm.nih.gov/compound/70


In [123]:
df['neutralized_pubchem_urls'] = df.metatlas_inchi_key.apply(get_pubchem_url_from_inchikey)


In [125]:
df.head()

,cid,inchi,charge,formula,monoisotopic_mw,M+proton,M+ammonium,M+sodium,M-proton,M+acetate,M+chloride,num_radicals,metatlas_inchi,metatlas_inchi_key,flat_inchi,flat_inchikey,pubchem_url_as_given,neutralized_pubchem_urls
0,19.0,InChI=1S/C7H6O4/c8-5-3-1-2-4(6(5)9)7(10)11/h1-...,0,C7H6O4,154.026609,155.033885,172.060432,177.015827,153.019333,213.040460,188.996011,0,InChI=1S/C7H6O4/c8-5-3-1-2-4(6(5)9)7(10)11/h1-...,GLDQAMYCGOIJDV-UHFFFAOYSA-N,InChI=1S/C7H6O4/c8-5-3-1-2-4(6(5)9)7(10)11/h1-...,GLDQAMYCGOIJDV-UHFFFAOYSA-N,https://pubchem.ncbi.nlm.nih.gov/compound/19,https://pubchem.ncbi.nlm.nih.gov/compound/19
1,47.0,"InChI=1S/C6H10O3/c1-3-4(2)5(7)6(8)9/h4H,3H2,1-...",0,C6H10O3,130.062994,131.070270,148.096817,153.052212,129.055718,189.076845,165.032396,0,"InChI=1S/C6H10O3/c1-3-4(2)5(7)6(8)9/h4H,3H2,1-...",JVQYSWDUAOAHFM-UHFFFAOYSA-N,"InChI=1S/C6H10O3/c1-3-4(2)5(7)6(8)9/h4H,3H2,1-...",JVQYSWDUAOAHFM-UHFFFAOYSA-N,https://pubchem.ncbi.nlm.nih.gov/compound/47,https://pubchem.ncbi.nlm.nih.gov/compound/47
2,51.0,"InChI=1S/C5H6O5/c6-3(5(9)10)1-2-4(7)8/h1-2H2,(...",0,C5H6O5,146.021523,147.028799,164.055346,169.010741,145.014247,205.035374,180.990925,0,"InChI=1S/C5H6O5/c6-3(5(9)10)1-2-4(7)8/h1-2H2,(...",KPGXRSRHYNQIFN-UHFFFAOYSA-N,"InChI=1S/C5H6O5/c6-3(5(9)10)1-2-4(7)8/h1-2H2,(...",KPGXRSRHYNQIFN-UHFFFAOYSA-N,https://pubchem.ncbi.nlm.nih.gov/compound/51,https://pubchem.ncbi.nlm.nih.gov/compound/51
3,58.0,"InChI=1S/C4H6O3/c1-2-3(5)4(6)7/h2H2,1H3,(H,6,7)",0,C4H6O3,102.031694,103.038970,120.065517,125.020912,101.024418,161.045545,137.001096,0,"InChI=1S/C4H6O3/c1-2-3(5)4(6)7/h2H2,1H3,(H,6,7)",TYEYBOSBBBHJIV-UHFFFAOYSA-N,"InChI=1S/C4H6O3/c1-2-3(5)4(6)7/h2H2,1H3,(H,6,7)",TYEYBOSBBBHJIV-UHFFFAOYSA-N,https://pubchem.ncbi.nlm.nih.gov/compound/58,https://pubchem.ncbi.nlm.nih.gov/compound/58
4,70.0,"InChI=1S/C6H10O3/c1-4(2)3-5(7)6(8)9/h4H,3H2,1-...",0,C6H10O3,130.062994,131.070270,148.096817,153.052212,129.055718,189.076845,165.032396,0,"InChI=1S/C6H10O3/c1-4(2)3-5(7)6(8)9/h4H,3H2,1-...",BKAJNAXTPSGJCU-UHFFFAOYSA-N,"InChI=1S/C6H10O3/c1-4(2)3-5(7)6(8)9/h4H,3H2,1-...",BKAJNAXTPSGJCU-UHFFFAOYSA-N,https://pubchem.ncbi.nlm.nih.gov/compound/70,https://pubchem.ncbi.nlm.nih.gov/compound/70


In [80]:
df.loc[37,:]

cid                                                                 247
inchi                     InChI=1S/C5H11NO2/c1-6(2,3)4-5(7)8/h4H2,1-3H3
charge                                                                1
formula                                                       C5H12NO2+
monoisotopic_mw                                                 118.086
M+proton                                                        119.094
M+ammonium                                                       136.12
M+sodium                                                        141.075
M-proton                                                        117.079
M+acetate                                                         177.1
M+chloride                                                      153.056
num_radicals                                                          0
metatlas_inchi        InChI=1S/C5H11NO2/c1-6(2,3)4-5(7)8/h4H2,1-3H3/p+1
metatlas_inchi_key                          KWIUHFFTVRNATP-UHFFF

In [74]:
m = Chem.MolFromInchi(str(df.loc[37,'inchi']))
print Descriptors.ExactMolWt(m)
m2 = desalt_compounds_in_dataframe(m)
print Descriptors.ExactMolWt(m2)
m3,b = import_compounds.NeutraliseCharges(m2)
m3
print Descriptors.ExactMolWt(m3)

117.078978592
117.078978592
118.086255044


In [42]:
df.head()

,cid,inchi,charge,formula,monoisotopic_mw,M+proton,M+ammonium,M+sodium,M-proton,M+acetate,M+chloride,num_radicals,metatlas_inchi,metatlas_inchi_key,flat_inchi,flat_inchikey
0,19.0,InChI=1S/C7H6O4/c8-5-3-1-2-4(6(5)9)7(10)11/h1-...,0.0,C7H6O4,154.026609,155.033885,172.060432,177.015827,153.019333,213.040460,188.996011,0.0,InChI=1S/C7H6O4/c8-5-3-1-2-4(6(5)9)7(10)11/h1-...,GLDQAMYCGOIJDV-UHFFFAOYSA-N,InChI=1S/C7H6O4/c8-5-3-1-2-4(6(5)9)7(10)11/h1-...,GLDQAMYCGOIJDV-UHFFFAOYSA-N
1,47.0,"InChI=1S/C6H10O3/c1-3-4(2)5(7)6(8)9/h4H,3H2,1-...",0.0,C6H10O3,130.062994,131.070270,148.096817,153.052212,129.055718,189.076845,165.032396,0.0,"InChI=1S/C6H10O3/c1-3-4(2)5(7)6(8)9/h4H,3H2,1-...",JVQYSWDUAOAHFM-UHFFFAOYSA-N,"InChI=1S/C6H10O3/c1-3-4(2)5(7)6(8)9/h4H,3H2,1-...",JVQYSWDUAOAHFM-UHFFFAOYSA-N
2,51.0,"InChI=1S/C5H6O5/c6-3(5(9)10)1-2-4(7)8/h1-2H2,(...",0.0,C5H6O5,146.021523,147.028799,164.055346,169.010741,145.014247,205.035374,180.990925,0.0,"InChI=1S/C5H6O5/c6-3(5(9)10)1-2-4(7)8/h1-2H2,(...",KPGXRSRHYNQIFN-UHFFFAOYSA-N,"InChI=1S/C5H6O5/c6-3(5(9)10)1-2-4(7)8/h1-2H2,(...",KPGXRSRHYNQIFN-UHFFFAOYSA-N
3,58.0,"InChI=1S/C4H6O3/c1-2-3(5)4(6)7/h2H2,1H3,(H,6,7)",0.0,C4H6O3,102.031694,103.038970,120.065517,125.020912,101.024418,161.045545,137.001096,0.0,"InChI=1S/C4H6O3/c1-2-3(5)4(6)7/h2H2,1H3,(H,6,7)",TYEYBOSBBBHJIV-UHFFFAOYSA-N,"InChI=1S/C4H6O3/c1-2-3(5)4(6)7/h2H2,1H3,(H,6,7)",TYEYBOSBBBHJIV-UHFFFAOYSA-N
4,70.0,"InChI=1S/C6H10O3/c1-4(2)3-5(7)6(8)9/h4H,3H2,1-...",0.0,C6H10O3,130.062994,131.070270,148.096817,153.052212,129.055718,189.076845,165.032396,0.0,"InChI=1S/C6H10O3/c1-4(2)3-5(7)6(8)9/h4H,3H2,1-...",BKAJNAXTPSGJCU-UHFFFAOYSA-N,"InChI=1S/C6H10O3/c1-4(2)3-5(7)6(8)9/h4H,3H2,1-...",BKAJNAXTPSGJCU-UHFFFAOYSA-N


In [126]:
df.to_csv('msmls_annotated.csv',index=None)

In [ ]:
# pool = mp.Pool(processes=20)
# pool.map(write_pubchem_info_to_file, inchi_keys[1:100])

In [ ]:
url = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/inchikey/%s/synonyms/json'%'CKLJMWTZIZZHCS-REOHCLBHSA-N'
response = requests.get(url)
response.json()

In [ ]:
response.json()

In [ ]:
url = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/%s/json'%'Baumin'
response = requests.get(url)
response.json()